In [3]:
# Build a Neuron Metadata Table from MATLAB Data
# -----------------------------------------------------------
# This script loads neuron spike timing and session metadata from .mat files,
# aligns it all together, and outputs a table with one row per neuron.

import scipy.io
import numpy as np
import pandas as pd

In [4]:
# === STEP 1: Load MATLAB Files ===
# These files contain info about sessions, brain areas, neuron-session mappings, and spike times
sessions_data = scipy.io.loadmat("sessions.mat", simplify_cells=True)
areas_data = scipy.io.loadmat("areaCell.mat", simplify_cells=True)
timestamps_data = scipy.io.loadmat("timestampsOfCellAll.mat", simplify_cells=True)
vCell_data = scipy.io.loadmat("vCell.mat", simplify_cells=True)

In [5]:
# === STEP 2: Extract and Flatten Data ===
vCell_values = vCell_data['vCell'].flatten() - 1  # Convert MATLAB (1-based) to Python (0-based)
session_info = sessions_data['sessions']          # Already a list with simplify_cells=True
areas = areas_data['areaCell']
timestamps_all = timestamps_data['timestampsOfCellAll']

In [6]:
# === STEP 3: Prepare Lists for DataFrame ===
neuron_ids = []
subject_ids = []
session_ids = []
aligned_areas = []
spike_counts = []

In [7]:
# === STEP 4: Loop Over Each Neuron ===
for i in range(len(vCell_values)):
    neuron_ids.append(i)
    sess_idx = vCell_values[i]

    # Session ID and subject ID
    if 0 <= sess_idx < len(session_info):
        session_id = session_info[sess_idx]['ID']
        session_ids.append(session_id)
        subject_ids.append(session_id.split("_")[0])
    else:
        session_ids.append("Unknown")
        subject_ids.append("Unknown")

    # Area
    aligned_areas.append(areas[i] if i < len(areas) else "Unknown")

    # Spike count
    if i < len(timestamps_all) and isinstance(timestamps_all[i], np.ndarray):
        spike_counts.append(len(timestamps_all[i]))
    else:
        spike_counts.append(0)

In [8]:
# === STEP 5: Create Final DataFrame ===
spike_df = pd.DataFrame({
    'neuron_id': neuron_ids,
    'subject_id': subject_ids,
    'session_id': session_ids,
    'area': aligned_areas,
    'spike_count': spike_counts
})

In [9]:
spike_df

,neuron_id,subject_id,session_id,area,spike_count
0,0,p6WV,p6WV_CelebA_Sess1,LAH,383
1,1,p6WV,p6WV_CelebA_Sess1,LAH,100
2,2,p6WV,p6WV_CelebA_Sess1,LAH,7732
3,3,p6WV,p6WV_CelebA_Sess1,LAH,231
4,4,p6WV,p6WV_CelebA_Sess1,LAH,2461
...,...,...,...,...,...
2077,2077,p20WV,p20WV_CelebA_S3_FBI_S1,LA,298
2078,2078,p20WV,p20WV_CelebA_S3_FBI_S1,LA,1053
2079,2079,p20WV,p20WV_CelebA_S3_FBI_S1,LA,5130
2080,2080,p20WV,p20WV_CelebA_S3_FBI_S1,LA,4269


In [10]:
# === STEP 6: Save to CSV ===
spike_df.to_csv("spike_data.csv", index=False)